# Lab 1/2 EMG Processing

This lab will go into the basics of real-time classification using EMG signals. 

In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt

In [2]:
''' If you have pyqt installed, this command will pop out interactive windows for graphs'''
%matplotlib qt

# Myo Data Processing
The following cells are for EMG data processing from the sample file

In [3]:
# Lets read in our data
import os
directory = 'Data/'
path = 'p18_emg.csv'
myo_df = pd.read_csv(directory + path)
myo_df.columns = myo_df.columns.str.replace(' ', '')
myo_df = myo_df.groupby('Arm').get_group('left') # This only needs to be done if you have two Myos running at the same time
display(myo_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
0,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
1,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
3,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
4,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555959,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,-110,-117,-33,-12,False,0,0.117068,-0.861911,-3.114586
1555962,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-3,-118,-114,-28,-2,False,0,0.135021,-0.854758,-3.131782
1555963,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-1,-82,-128,-16,-2,False,0,0.135021,-0.854758,-3.131782
1555966,2291479052128,warm,True,left,2019-02-14 16:13:28 764500,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,12,-104,-119,-13,-1,False,0,0.135021,-0.854758,-3.131782


## Plot EMG and IMU Channels

In [4]:
''' Plot Entire EMG signals'''
for channel in range(1,9):
    plt.figure()
    ax = myo_df['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    myo_df['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')

## Get Descripitive Statistics for Raw Data

In [5]:
myo_df.describe()

,DeviceID,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,Acc_Y,Acc_Z,Gyro_X,Gyro_Y,...,EMG_3,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,RSSI,Roll,Pitch,Yaw
count,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,...,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1405884.0,1.405884e+06,1.405884e+06,1.405884e+06
mean,2.291479e+12,5.829820e-02,-1.416711e-01,4.756671e-02,7.029249e-02,7.194029e-01,2.172123e-02,4.590001e-01,4.715409e-01,2.128901e-02,...,-8.544162e-01,-7.970885e-01,-8.382363e-01,-1.035961e+00,-7.810957e-01,-7.580533e-01,0.0,9.882653e-02,-9.217510e-01,6.220871e-03
std,1.105763e+02,3.596622e-01,6.225795e-01,3.632931e-01,5.660227e-01,3.237783e-01,3.134963e-01,2.499457e-01,3.317542e+01,2.453375e+01,...,1.196934e+01,7.937094e+00,1.137876e+01,1.678093e+01,9.360004e+00,1.066679e+01,0.0,4.923939e-01,4.655549e-01,1.682120e+00
min,2.291479e+12,-7.495117e-01,-9.933472e-01,-8.003540e-01,-9.848633e-01,-1.368164e+00,-2.659668e+00,-1.334473e+00,-4.955625e+02,-5.199375e+02,...,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,0.0,-3.140510e+00,-1.570796e+00,-3.141558e+00
25%,2.291479e+12,-2.727051e-01,-7.298584e-01,-2.425537e-01,-4.491577e-01,5.043945e-01,-1.855469e-01,2.695312e-01,-6.375000e+00,-3.437500e+00,...,-3.000000e+00,-3.000000e+00,-3.000000e+00,-4.000000e+00,-3.000000e+00,-3.000000e+00,0.0,-2.744367e-01,-1.277161e+00,-1.384925e+00
50%,2.291479e+12,1.106567e-01,-2.828979e-01,9.576416e-02,9.210205e-02,9.023438e-01,4.101562e-02,3.520508e-01,1.875000e-01,0.000000e+00,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.0,1.528443e-01,-1.194552e+00,1.360732e-01
75%,2.291479e+12,3.458252e-01,4.904785e-01,3.248901e-01,6.166992e-01,9.409180e-01,1.313477e-01,7.158203e-01,6.750000e+00,3.437500e+00,...,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,0.0,4.110263e-01,-5.247361e-01,1.339452e+00
max,2.291479e+12,7.894287e-01,9.920654e-01,7.677612e-01,9.845581e-01,2.842773e+00,1.588867e+00,1.886230e+00,5.243125e+02,4.203125e+02,...,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,0.0,3.139911e+00,1.480952e+00,3.141591e+00


## Rectify the Signal

In [6]:
rectified_df =myo_df.copy() # make copy of DF

for col in ['EMG_' + str(i) for i in range(1, 9)]:
    rectified_df[col] = rectified_df[col].apply(abs) # applys the absolute function to each channel
    
display(rectified_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
0,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,2,1,1,1,1,False,0,0.398451,-1.233521,1.567341
1,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,3,1,1,0,0,False,0,0.398451,-1.233521,1.567341
2,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,1,2,1,1,False,0,0.398451,-1.233521,1.567341
3,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,1,2,1,False,0,0.398451,-1.233521,1.567341
4,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555959,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,110,117,33,12,False,0,0.117068,-0.861911,-3.114586
1555962,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,3,118,114,28,2,False,0,0.135021,-0.854758,-3.131782
1555963,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,1,82,128,16,2,False,0,0.135021,-0.854758,-3.131782
1555966,2291479052128,warm,True,left,2019-02-14 16:13:28 764500,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,12,104,119,13,1,False,0,0.135021,-0.854758,-3.131782


### Plot the Rectified Signals and Look at the new Means/Stdevs for each EMG Channel

In [7]:
'''Your Code Here'''

for channel in range(1,9):
    plt.figure()
    ax = rectified_df['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    rectified_df['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')
'''Stop Coding Here'''

C:\Users\joewp\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


'Stop Coding Here'

### Apply a rolling average with a window size of 400 (2 seconds) and look at the new plots
Look into Pandas rolling method (e.g, myo_df.rolling(400).mean())

In [8]:
'''Your code here'''
rectified_df.rolling(400).mean()
for channel in range(1,9):
    plt.figure()
    ax = rectified_df.rolling(400).mean()['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    rectified_df.rolling(400).mean()['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')
'''Stop coding here'''

'Stop coding here'

### Advanced EMG Filtering
Now, we are going to apply a bandpass filter to each EMG channel.

In [41]:
import scipy as sp
import scipy.signal

def filteremg(emg, low_pass=3, sfreq=200, high_band=20, low_band=95):
    """
    emg: EMG data
    high: high-pass cut off frequency
    low: low-pass cut off frequency
    sfreq: sampling frequency
    """
    # Zero mean emg signal
    emg = emg - emg.mean()
    
    # normalise cut-off frequencies to sampling frequency
    high_band = high_band/(sfreq/2)
    low_band = low_band/(sfreq/2)
    
    
    # create bandpass filter for EMG
    b1, a1 = sp.signal.butter(4, [high_band,low_band], btype='bandpass', analog=True)
    
    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b1, a1, emg)    
    
    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)
    
    # create lowpass filter and apply to rectified signal to get EMG envelope
    low_pass = low_pass/(sfreq/2)
    b2, a2 = sp.signal.butter(4, low_pass, fs=sfreq, btype='lowpass')
    emg_envelope = sp.signal.lfilter(b2, a2, emg_rectified)
    
    return emg_rectified
    

filt_emg = myo_df.copy()
emg_keys = ['EMG_' + str(i) for i in range(1, 9)]
filt_emg[emg_keys] = filt_emg[emg_keys].apply(filteremg, raw=True)
display(filt_emg)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0.461470,0.079886,4.582753,0.963620,0.386037,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,5.011933,1.036208,1.203200,2.946154,4.593091,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,2.244083,0.732076,7.114925,1.487094,2.558975,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,3.155058,0.879967,4.930477,3.851222,3.223841,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.756350,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,2.771569,1.728994,3.497388,5.094889,3.925478,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-14 16:13:28.749540,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,24.053377,120.951503,5.786276,35.171811,11.832786,False,0,0.117068,-0.861911,-3.114586
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,30.107308,75.708617,3.280509,8.586359,3.486051,False,0,0.135021,-0.854758,-3.131782
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,13.793693,78.803861,18.638608,25.451561,6.374921,False,0,0.135021,-0.854758,-3.131782


## Plot Filtered Signals and Get Mean/Stdev

In [10]:
''' Your code here'''

for channel in range(1,9):
    plt.figure()
    ax = filt_emg['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    filt_emg['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')

''' Stop coding here'''

' Stop coding here'

## Power Spectral Density
Now, we are going to look at a PSF plot

In [11]:
f, Pxx_den = sp.signal.periodogram(filt_emg['EMG_1'], 200)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

### Plot the PSD for each filtered EMG channel and Find the Max Power

In [12]:
''' Your Code Here'''
max_power_list = []
for channel in range(1,9):
    plt.figure()
    f, Pxx_den = sp.signal.periodogram(filt_emg['EMG_'+ str(channel)], 200)
    plt.semilogy(f, Pxx_den)
    plt.ylim([1e-7, 1e2])
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.show()
    max_power_list.append(max(Pxx_den))
display(max_power_list)
''' Stop Coding Here'''

[31135.821243653016,
 59306.656347168384,
 72921.52358036177,
 25172.49760987416,
 37437.66307947198,
 89367.2954260323,
 43524.42992066737,
 41144.892392411006]

' Stop Coding Here'

# Segmenting Data
Using your collected timestamps, we are now going to look at each gesture: rock, paper, scissors

In [13]:
'''First, we need to set pandas indexes to timestamps'''
myo_df.index = pd.to_datetime(myo_df['Timestamp'], format='%Y-%m-%d %H:%M:%S %f' )
display(myo_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.756350,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-14 16:13:28.749540,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,-110,-117,-33,-12,False,0,0.117068,-0.861911,-3.114586
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-3,-118,-114,-28,-2,False,0,0.135021,-0.854758,-3.131782
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-1,-82,-128,-16,-2,False,0,0.135021,-0.854758,-3.131782


In [14]:
'''Then we can segment data using the indexes'''
start = pd.to_datetime('2019-02-14 14:46:53')
stop  = pd.to_datetime('2019-02-14 14:48:53')
gesture_df = myo_df[start:stop]
display(gesture_df)
''' Notice that we did not need to specify the nano seconds'''

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.756350,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-14 14:47:27.604011,2291479052128,warm,True,left,2019-02-14 14:47:27 604011,0.150879,-0.792053,-0.540100,-0.241394,-1.056641,...,-4,36,-4,3,25,False,0,0.058538,1.190134,0.631412
2019-02-14 14:47:27.619038,2291479052128,warm,True,left,2019-02-14 14:47:27 619038,0.151001,-0.792786,-0.540466,-0.238037,-1.041992,...,-1,10,1,23,39,False,0,0.048260,1.191257,0.616196
2019-02-14 14:47:27.619038,2291479052128,warm,True,left,2019-02-14 14:47:27 619038,0.151001,-0.792786,-0.540466,-0.238037,-1.041992,...,3,-24,0,17,42,False,0,0.048260,1.191257,0.616196


' Notice that we did not need to specify the nano seconds'

## Reading in XDF
Now we are going to read in from an XDF file

In [15]:
''' Imports and useful functions'''
import pyxdf

def xdf_to_dataframe(xdf_data):
    ''' Xdf Data should be a list of streams (dictionaries)
        Function returns a dictionary of dataframes, one dataframe per stream'''  
    dataframes = {}
    for stream in xdf_data:
        df = pd.DataFrame()
        data = stream['time_series']
        timestamps = stream['time_stamps']
        df['Time'] = timestamps
        chan_names, units = get_channel_names(stream['info'])
        counts = data.shape[0]
        for series, name, unit in zip(range(data.shape[1]), chan_names, units):
            df[name[0]]  = data[:, series]
            if unit:
                df[name[0] + '_Unit'] = np.repeat(unit, counts)
                
        
        for item in stream['info']:
            if item not in ['name', 'desc', 'data']:
                try:
                    df[item] = np.repeat(stream['info'][item], counts)
                except:
                    continue
        dataframes[stream['info']['name'][0]] = df
        
    return dataframes
            
        
        

def get_channel_names(info):
    channels = info['desc'][0]['channels'][0]['channel']
    names = [chan['label'] for chan in channels ]
    units = [chan['unit'] for chan in channels ]
    return names, units

In [16]:
data, header = pyxdf.load_xdf('Data/test.xdf')
dfs = xdf_to_dataframe(data)
display(dfs['BioRadio-20312'])
''' Save new dataframe'''
dfs['BioRadio-20312'].to_csv('Data/test.csv')

,Time,CH1,CH1_Unit,CH2,CH2_Unit,type,channel_count,nominal_srate,channel_format,source_id,...,session_id,hostname,v4address,v4data_port,v4service_port,v6address,v6data_port,v6service_port,stream_id,effective_srate
0,6998.573759,24.987793,mV,24.987793,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
1,6998.577665,-0.085449,mV,-0.048828,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
2,6998.581571,24.987793,mV,24.987793,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
3,6998.585477,-0.073242,mV,-0.036621,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
4,6998.589383,24.987793,mV,24.987793,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,7006.288357,0.122070,mV,0.000000,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
1976,7006.292263,-25.000000,mV,-25.000000,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
1977,7006.296169,0.073242,mV,-0.012207,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134
1978,7006.300076,-25.000000,mV,-25.000000,mV,RAW,2,256,float32,20312,...,default,GLE-1000-PC06,None,16572,16572,None,16572,16572,1,256.008134


# Your turn!
The above code introduced you to some simple EMG processing from already collected data, but now lets use your data.

For the Myo and BioRadio, do the following for each EMG channel:
- Import the files
- Segment the data for one Rock, Paper, and Scissors gesture based on your collected timestamps
- Filter each gesture using a bandpass filter and a notch filter (at 60 hz)
- Plot the Filtered EMG signals and the Power Spectral Density of each Gesture
- Determine the Max Power for each Gesture
- Determine the mean/standard deviation for each Gesture

Your report should include each plot in an appendix.

In the main body of your report, include a table for each gesture that has columns: Device, EMG Channel Number, Mean, Std Dev, Max Power. Fill out the rows

In [118]:
from scipy import signal

data, header = pyxdf.load_xdf('Data/Myo/rock3.xdf')
dfs = xdf_to_dataframe(data)
print(dfs.keys())
display(dfs['Thalmic Labs MyoMyo'])
'''Save new dataframe'''
dfs['Thalmic Labs MyoMyo'].to_csv('Data/paper1.csv')

filt_BP = dfs['Thalmic Labs MyoMyo'].copy()
BP_keys = ['EMG_' + str(i) for i in range(1, 9)]
filt_BP[BP_keys] = filt_BP[BP_keys].apply(filteremg, raw=True)
plt.close('all')
filt_notch = filt_BP.copy()
b, a = signal.iirnotch(60, 3, 256)
print(b)
print(a)

for channel in range(1,9):
    filt_notch['EMG_'+str(channel)] = signal.filtfilt(b, a, filt_notch['EMG_'+str(channel)])

plt.figure()   
for channel2 in range(1,9):
    ax = filt_notch['EMG_'+str(channel2)].plot()
    print(('CH' + str(channel2) +  ' Mean:' + str(filt_notch['EMG_'+str(channel2)].mean())+ ' STDev:' + str(filt_notch['EMG_'+str(channel2)].std())))
plt.title('Notch Filter')
plt.ylabel('mVolts')
plt.xlabel('Time')
plt.legend(range(1,9))
    
#for channel in range(1,5):
    #plt.figure()
    #ax = filt_BP['CH' + str(channel)].plot()
    #plt.title('CH' + str(channel))
    #plt.ylabel('mVolts')
    #plt.xlabel('Time')

max_power_list = []
plt.figure()
for channel in range(1,9):
    f, Pxx_den = sp.signal.periodogram(filt_notch['EMG_'+ str(channel)], 256)
    plt.semilogy(f, Pxx_den)
    plt.ylim([1e-7, 1e2])
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.show()
    max_power_list.append(max(Pxx_den))
plt.title('Power Spectral Density')
plt.legend(range(1,9))
display(max_power_list)


dict_keys(['Thalmic Labs MyoMyo'])


,Time,Device ID,Device ID_Unit,Warm?,Sync,Arm,Arm_Unit,Timestamp,Timestamp_Unit,Orientation_W,...,session_id,hostname,v4address,v4data_port,v4service_port,v6address,v6data_port,v6service_port,stream_id,effective_srate
0,1347.580608,1.431245e+12,Number,0.0,1.0,1.0,Arm,1347.589722,Time,0.828552,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
1,1347.585621,1.431245e+12,Number,0.0,1.0,1.0,Arm,1347.590088,Time,0.828552,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
2,1347.590633,1.431245e+12,Number,0.0,1.0,1.0,Arm,1347.590698,Time,0.828491,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
3,1347.595646,1.431245e+12,Number,0.0,1.0,1.0,Arm,1347.590942,Time,0.828491,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
4,1347.600658,1.431245e+12,Number,0.0,1.0,1.0,Arm,1347.603027,Time,0.828491,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1350.242171,1.431245e+12,Number,0.0,1.0,1.0,Arm,1350.236938,Time,0.828003,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
532,1350.247183,1.431245e+12,Number,0.0,1.0,1.0,Arm,1350.250366,Time,0.827637,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
533,1350.252196,1.431245e+12,Number,0.0,1.0,1.0,Arm,1350.250610,Time,0.827637,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787
534,1350.257208,1.431245e+12,Number,0.0,1.0,1.0,Arm,1350.250977,Time,0.827637,...,default,MICHAELLAPTOP,None,16573,16573,None,16573,16573,2,199.507787


[ 0.79968847 -0.15676635  0.79968847]
[ 1.         -0.15676635  0.59937693]
CH1 Mean:37.281125736819924 STDev:60.008898225298815
CH2 Mean:34.0456360994981 STDev:43.004755144044175
CH3 Mean:21.617625735774457 STDev:32.72140808101125
CH4 Mean:41.73791227536002 STDev:64.5521825545476
CH5 Mean:37.16760324096015 STDev:41.42932373542463
CH6 Mean:26.356342267126642 STDev:32.60668234411674
CH7 Mean:48.15018929843469 STDev:74.3387686029987
CH8 Mean:35.54728338057643 STDev:52.302888405383314


[3171.416504629567,
 1646.7511634984237,
 864.7432898532326,
 3806.5437928291744,
 1294.3044117595439,
 849.3055790832778,
 4967.993213662543,
 2477.5310630727813]